In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, GRU,SimpleRNN
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import BatchNormalization
# from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

from sklearn.model_selection import train_test_split
from ast import literal_eval

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

2024-02-09 21:26:16.416420: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 21:26:16.416549: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 21:26:16.561489: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:

train_df = pd.read_csv('/kaggle/input/kriti2024/train.csv')

In [3]:
train_df['CONTEXT'] = train_df['Title'] + ". " + train_df['Abstract']
train_df.drop(labels=['Title', 'Abstract', 'Id'], axis=1, inplace=True)
train_df

,Categories,CONTEXT
0,['cs.LO'],Axiomatic Aspects of Default Inference. This p...
1,['math.GR'],On extensions of group with infinite conjugacy...
2,"['cs.LG', 'cs.IT', 'eess.SP', 'math.IT']",An Analysis of Complex-Valued CNNs for RF Data...
3,"['math.PR', 'math.ST', 'stat.TH']",On the reconstruction of the drift of a diffus...
4,"['cs.IT', 'math.IT']",Three classes of propagation rules for GRS and...
...,...,...
51205,['math.AP'],Generalized Fourier Integral Operators on spac...
51206,"['cs.CV', 'cs.CL']",Weakly-Supervised 3D Visual Grounding based on...
51207,['math.CV'],Strongly pseudoconvex handlebodies. We give an...
51208,"['cs.CL', 'cs.LG', 'cs.SD', 'eess.AS']",Improving End-to-End Speech Processing by Effi...


In [4]:
max_seqlen = 250
batch_size = 256
padding_token = "<pad>"
auto = tf.data.AUTOTUNE

In [5]:
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

def decontract(sentence):
    # specific
    sentence = re.sub(r"won't", "will not", sentence)
    sentence = re.sub(r"can\'t", "can not", sentence)

    # general
    sentence = re.sub(r"n\'t", " not", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'s", " is", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'t", " not", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'m", " am", sentence)
    return sentence

def cleanPunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', '', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub("", sentence)

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

re_stop_words = re.compile(r"\b(" + "|".join(stopwords) + ")\\W", re.I)

stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

train_df['CONTEXT']=train_df['CONTEXT'].str.lower()
train_df['CONTEXT']=train_df['CONTEXT'].apply(decontract)
train_df['CONTEXT']=train_df['CONTEXT'].apply(cleanPunc)
train_df['CONTEXT']=train_df['CONTEXT'].apply(removeStopWords)
train_df['CONTEXT']=train_df['CONTEXT'].apply(keepAlpha)

In [6]:
# import nltk
# nltk.download('punkt')
# from nltk.tokenize import word_tokenize
# from collections import Counter


# # Tokenize the sentences into words and create a list of all words
# all_words = []
# for sentence in train_df['CONTEXT']:
#     words = word_tokenize(sentence)
#     all_words.extend(words)

# # Count the frequency of each word
# word_freq = Counter(all_words)

# # Get the top 10 words
# top_10_words = word_freq.most_common(10)

# # Convert the result to a DataFrame for easier display
# top_10_words_df = pd.DataFrame(top_10_words, columns=['Word', 'Frequency'])

# # Display the top 10 words and their frequencies
# print(top_10_words_df)

In [7]:
# def remove_top_words(sentence):
#     words = word_tokenize(sentence)
#     words = [word for word in words if word not in top_10_words]
#     return ' '.join(words)

In [8]:

# train_df['CONTEXT']=train_df['CONTEXT'].apply(remove_top_words)
# train_df['CONTEXT']

In [9]:

train_df.reset_index(inplace = True)
train_df['Categories'] = train_df['Categories'].str.split(', ' , expand = True)


test_df  = pd.read_csv('/kaggle/input/kriti2024/test.csv')

#drop off the categories with 1 occurence
train_df_filtered = train_df.groupby("Categories").filter(lambda x: len(x) > 1)


train_df_filtered["Categories"] = train_df_filtered["Categories"].apply(
    lambda x: literal_eval(x)
)

test_split = 0.2

train_df, test_df = train_test_split(
    train_df_filtered,
    test_size=test_split,
    stratify=train_df_filtered["Categories"].values,
)

# Splitting the test set further into validation
# and new test sets.
val_df = test_df.sample(frac=0.5)
test_df.drop(val_df.index, inplace=True)

print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in validation set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

terms = tf.ragged.constant(train_df["Categories"].values)
lookup = tf.keras.layers.StringLookup(output_mode="multi_hot")
lookup.adapt(terms)
vocab = lookup.get_vocabulary()


def invert_multi_hot(encoded_labels):
    """Reverse a single multi-hot encoded label to a list of vocab terms."""
    hot_indices = np.where(encoded_labels > 0.4)[0]
    terms = np.take(vocab, hot_indices, mode='clip')  # Use 'clip' mode to handle out-of-bounds indices
    return terms.tolist()


print(len(vocab))




def make_dataset(dataframe, is_train=True):
    labels = tf.ragged.constant(dataframe["Categories"].values)
    label_binarized = lookup(labels).numpy()
    dataset = tf.data.Dataset.from_tensor_slices(
        (dataframe["CONTEXT"].values, label_binarized)
    )
    dataset = dataset.shuffle(batch_size * 10) if is_train else dataset
    return dataset.batch(batch_size)

train_dataset = make_dataset(train_df, is_train=True)
validation_dataset = make_dataset(val_df, is_train=False)
test_dataset = make_dataset(test_df, is_train=False)

text_batch, label_batch = next(iter(train_dataset))

for i, text in enumerate(text_batch[:5]):
    label = label_batch[i].numpy()[None, ...]
    print(f"Abstract: {text}")
    print(f"Label(s): {invert_multi_hot(label[0])}")
    print(" ")

vocabulary = set()
train_df["CONTEXT"].str.lower().str.split().apply(vocabulary.update)
# train_df["Abstract"].str.lower().str.split().apply(vocabulary.update)
vocabulary_size = len(vocabulary)
print(vocabulary_size)

text_vectorizer = layers.TextVectorization(
    max_tokens=vocabulary_size, ngrams=2, output_mode="tf_idf"
)

# `TextVectorization` layer needs to be adapted as per the vocabulary from our
# training set.
with tf.device("/CPU:0"):
    text_vectorizer.adapt(train_dataset.map(lambda text, label: text))

train_dataset = train_dataset.map(
    lambda text, label: (text_vectorizer(text), label), num_parallel_calls=auto
).prefetch(auto)
validation_dataset = validation_dataset.map(
    lambda text, label: (text_vectorizer(text), label), num_parallel_calls=auto
).prefetch(auto)
test_dataset = test_dataset.map(
    lambda text, label: (text_vectorizer(text), label), num_parallel_calls=auto
).prefetch(auto)

train_batch,label = next(iter(train_dataset))
print(train_batch.shape, label.shape)

Number of rows in training set: 40495
Number of rows in validation set: 5062
Number of rows in test set: 5062
58
Abstract: b'characterization modular units provide exact formula complex exponents modular product expansion modular units deduce characterization modular units terms growth exponents answering question w kohnen'
Label(s): ['math.NT']
 
Abstract: b'performance analysis publish subscribe systems desktop grid offers solutions overcome several challenges answer increasingly needs scientific computing technology consists mainly exploiting resources geographically dispersed treat complex applications needing big power calculation important storage capacity however resources number increases need scalability selforganisation dynamic reconfigurations decentralisation performance becomes essential since properties exhibited pp systems convergence grid computing pp computing seems natural context paper evaluates scalability performance pp tools discovering registering services three 

In [10]:

train_df

,index,Categories,CONTEXT
34122,34122,[math.LO],strong measure zero subsets kappa paper answe...
21911,21911,[math.NT],moments rank elliptic curves fix elliptic curv...
12243,12243,[cs.LG],costar improved temporal counterfactual estima...
9945,9945,"[cs.LG, cs.AI, cs.CL]",language control diffusion efficiently scaling...
33766,33766,"[math.CO, math.AC]",generic initial ideals exterior algebraic shif...
...,...,...,...
2780,2780,"[cs.CV, cs.AI, cs.LG]",dual cognitive architecture incorporating bias...
14348,14348,"[stat.ME, math.ST, stat.CO, stat.TH]",bayesian estimation bivariate copula using jef...
33834,33834,"[cs.LG, cs.AI]",pde enhancing generalization via pde adaptive ...
42292,42292,[cs.CR],centralized intermediation decentralized web e...


In [11]:
K = keras.backend
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):

    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [12]:
%%time

# import keras.optimizers
initializer = tf.keras.initializers.HeUniform( seed = 223)
initializer2 = tf.keras.initializers.HeNormal( seed = 123)
final_initializer = tf.keras.initializers.GlorotUniform(seed = 425)
def make_model():
    shallow_mlp_model = keras.Sequential(
        [
            layers.Dense(1024, kernel_initializer = initializer,activation="swish"),
            layers.BatchNormalization(),
            layers.Dropout(0.6),
            layers.Dense(512,kernel_initializer = initializer,  activation="swish"),
            # layers.BatchNormalization(),
            layers.Dropout(0.5),
            layers.Dense(256,kernel_initializer = initializer, activation="swish"),
            # layers.BatchNormalization(),
            layers.Dropout(0.2),
            layers.Dense(128, kernel_initializer = initializer2, activation="leaky_relu"),
            layers.Dense(lookup.vocabulary_size(),kernel_initializer = final_initializer, activation="sigmoid"),
        ]
    )
    return shallow_mlp_model

epochs = 10

shallow_mlp_model = make_model()
shallow_mlp_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy",  Precision(), Recall()]
)

# shallow_mlp_model.compile(
#     loss=HammingLoss(mode='multilabel'),
#     optimizer="adam",
#     metrics=["accuracy", "binary_accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
# )
history = shallow_mlp_model.fit(
    train_dataset, validation_data=validation_dataset, epochs=epochs
)


Epoch 1/10
159/159 [==============================] - 362s 2s/step - loss: 0.1344 - accuracy: 0.3103 - precision: 0.3573 - recall: 0.2786 - val_loss: 0.0934 - val_accuracy: 0.5247 - val_precision: 0.5906 - val_recall: 0.7096
Epoch 2/10
159/159 [==============================] - 348s 2s/step - loss: 0.0628 - accuracy: 0.5139 - precision: 0.7654 - recall: 0.5346 - val_loss: 0.0612 - val_accuracy: 0.5367 - val_precision: 0.6805 - val_recall: 0.7067
Epoch 3/10
159/159 [==============================] - 348s 2s/step - loss: 0.0449 - accuracy: 0.5879 - precision: 0.8260 - recall: 0.6728 - val_loss: 0.0542 - val_accuracy: 0.5620 - val_precision: 0.7132 - val_recall: 0.7213
Epoch 4/10
159/159 [==============================] - 358s 2s/step - loss: 0.0347 - accuracy: 0.6268 - precision: 0.8626 - recall: 0.7560 - val_loss: 0.0554 - val_accuracy: 0.5739 - val_precision: 0.7081 - val_recall: 0.7301
Epoch 5/10
159/159 [==============================] - 348s 2s/step - loss: 0.0283 - accuracy: 0.6500

In [13]:
# Evaluate the model
test_loss, test_bin_acc, test_precision, test_recall = shallow_mlp_model.evaluate(test_dataset)
print(f'  Test Loss: {test_loss:.4f}')
print(f"Test binary accuracy: {test_bin_acc:.4f}")
print(f'Test Precision: {test_precision:.4f}, Test Recall: {test_recall:.4f}')
f1 = (2 * test_precision * test_recall) / (test_precision + test_recall)

print(f1)

20/20 [==============================] - 12s 566ms/step - loss: 0.0727 - accuracy: 0.5551 - precision: 0.7284 - recall: 0.7328
  Test Loss: 0.0727
Test binary accuracy: 0.5551
Test Precision: 0.7284, Test Recall: 0.7328
0.7305707205916312


In [14]:
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

test_df = pd.read_csv('/kaggle/input/kriti2024/test.csv')

def decontract(sentence):
    # specific
    sentence = re.sub(r"won't", "will not", sentence)
    sentence = re.sub(r"can\'t", "can not", sentence)

    # general
    sentence = re.sub(r"n\'t", " not", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'s", " is", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'t", " not", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'m", " am", sentence)
    return sentence

def cleanPunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', '', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub("", sentence)

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

re_stop_words = re.compile(r"\b(" + "|".join(stopwords) + ")\\W", re.I)

stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

test_df['CONTEXT'] = test_df['Title'] + ". " + test_df['Abstract']
test_df.drop(labels=['Title', 'Abstract'], axis=1, inplace=True)

test_df['CONTEXT']=test_df['CONTEXT'].str.lower()
test_df['CONTEXT']=test_df['CONTEXT'].apply(decontract)
test_df['CONTEXT']=test_df['CONTEXT'].apply(cleanPunc)
test_df['CONTEXT']=test_df['CONTEXT'].apply(removeStopWords)
test_df['CONTEXT']=test_df['CONTEXT'].apply(keepAlpha)

In [15]:
# test_df['CONTEXT']=test_df['CONTEXT'].apply(remove_top_words)
# test_df['CONTEXT']

In [16]:
#Predicitions

import pandas as pd
import numpy as np
from ast import literal_eval
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import TextVectorization

# Load the test dataset


# Preprocess the test dataset
test_df['Categories'] = np.nan  # Placeholder for predicted categories

#text_vectorizer.adapt(train_dataset.map(lambda text, label: text))
# Assuming train_df is your training dataframe with "Abstract" column
#text_vectorizer.adapt(tf.data.Dataset.from_tensor_slices(test_df["Abstract"].values).batch(batch_size))
# train_df["Abstract"].str.lower().str.split().apply(vocabulary.update)
# train_df["Title"].str.lower().str.split().apply(vocabulary.update)

test_dataset = tf.data.Dataset.from_tensor_slices(test_df["CONTEXT"].values )
test_dataset = test_dataset.map(lambda text: text_vectorizer(text)).batch(batch_size)

# Make predictions on the test dataset
predictions = shallow_mlp_model.predict(test_dataset)

# Convert predicted binary labels to categories using list comprehension
predicted_categories = [invert_multi_hot(row) for row in predictions]

# Convert the list of lists to a numpy array
predicted_categories = np.array(predicted_categories, dtype=object)



43/43 [==============================] - 31s 711ms/step


In [17]:
predicted_categories.shape

(10974,)

In [18]:
desired_column_order = [
    'Id', 'math.AT', 'stat.AP', 'cs.AR', 'math.QA', 'q-bio.MN', 'eess.AS', 'eess.IV', 'stat.ME',
    'econ.GN', 'eess.SP', 'q-fin.RM', 'cs.LG', 'cs.CR', 'q-bio.BM', 'q-fin.GN', 'q-fin.MF',
    'q-fin.PR', 'math.CV', 'cs.LO', 'econ.TH', 'math.CO', 'cs.AI', 'math.AC', 'q-bio.CB',
    'q-fin.CP', 'cs.CL', 'cs.DC', 'math.LO', 'math.NT', 'cs.SD', 'q-fin.TR', 'cs.CV',
    'stat.ML', 'q-fin.EC', 'econ.EM', 'cs.CE', 'stat.CO', 'math.PR', 'q-bio.NC', 'math.AP',
    'cs.OS', 'cs.NI', 'cs.IT', 'cs.PL', 'cs.GT', 'cs.DM', 'math.IT', 'cs.SE', 'cs.RO',
    'stat.TH', 'cs.DB', 'math.ST', 'q-bio.GN', 'q-fin.PM', 'q-bio.TO', 'math.GR', 'cs.IR'
]

In [19]:
# def extract_labels(sample):
#     """
#     Extract labels from the sample string and return them as a set.

#     Args:
#     - sample (str): The sample string containing labels separated by commas.

#     Returns:
#     - label_set (set): A set containing the extracted labels.
#     """
#     label_set = set(sample.split(','))
#     return label_set


# def multi_hot_encoding(labels, vocab):
#     """
#     Perform multi-hot encoding on the extracted labels based on the given vocabulary.

#     Args:
#     - labels (set): A set containing the extracted labels.
#     - vocab (set): The set of words in the vocabulary.

#     Returns:
#     - encoding (numpy array): The multi-hot encoding scheme for the labels.
#     """
#     encoding = [1 if label in labels else 0 for label in vocab]
#     return encoding

In [20]:
# encoded_labels = pd.DataFrame(columns=desired_column_order)
# encoded_labels["Id"] = test_df["Id"]


In [21]:

# predicted_categories = pd.DataFrame(predicted_categories)
# predicted_categories

In [22]:
# for category in encoded_labels.columns[1:]:
#     # Check if each category is present in the 'Categories' column
#     encoded_labels[category] = predicted_categories.apply(lambda x: 1 if category in x else 0)

In [23]:
# encoded_labels

In [24]:
predicted_categories[2]

['cs.LG', 'cs.CV', 'eess.IV']

In [25]:
from sklearn.preprocessing import MultiLabelBinarizer

# Create a MultiLabelBinarizer and fit on unique categories
mlb = MultiLabelBinarizer()
mlb.fit(predicted_categories)

# Transform the predicted_categories into binary representation
predicted_binary = mlb.transform(predicted_categories)

# Create the predicted_df DataFrame
predicted_df = pd.DataFrame(predicted_binary, columns=mlb.classes_, index=test_df.index)


# Concatenate the original test_df with the predicted_df
result_df = pd.concat([test_df, predicted_df], axis=1)

# Drop the 'Title' and 'Abstract' columns
result_df = result_df.drop(['CONTEXT','Categories'], axis=1)

# Save the results to a new CSV file
result_df.to_csv('submission.csv', index=False)


In [26]:
# class SimpleMultiLabelBinarizer:
#     def __init__(self):
#         self.classes_ = []

#     def fit(self, Y):
#         """Fit the binarizer to the unique categories in Y"""
#         unique_classes = set(cls for sublist in Y for cls in sublist)
#         self.classes_ = sorted(list(unique_classes))

#     def transform(self, Y):
#         """Transform the categories in Y to a binary matrix"""
#         binary_matrix = []
#         for labels in Y:
#             row = [int(cls in labels) for cls in self.classes_]
#             binary_matrix.append(row)
#         return binary_matrix

#     def fit_transform(self, Y):
#         """Fit to the data, then transform it"""
#         self.fit(Y)
#         return self.transform(Y)

In [27]:
predicted_categories

array([list(['q-fin.PR', 'q-fin.MF', 'q-fin.CP']), list(['cs.AR']),
       list(['cs.LG', 'cs.CV', 'eess.IV']), ..., list(['cs.CV']),
       list(['cs.LG', 'cs.AI', 'cs.CV', 'cs.RO']), list(['cs.LO'])],
      dtype=object)

In [28]:
import pandas as pd

predicted_df = pd.read_csv('submission.csv')

desired_column_order = [
    'Id', 'math.AT', 'stat.AP', 'cs.AR', 'math.QA', 'q-bio.MN', 'eess.AS', 'eess.IV', 'stat.ME',
    'econ.GN', 'eess.SP', 'q-fin.RM', 'cs.LG', 'cs.CR', 'q-bio.BM', 'q-fin.GN', 'q-fin.MF',
    'q-fin.PR', 'math.CV', 'cs.LO', 'econ.TH', 'math.CO', 'cs.AI', 'math.AC', 'q-bio.CB',
    'q-fin.CP', 'cs.CL', 'cs.DC', 'math.LO', 'math.NT', 'cs.SD', 'q-fin.TR', 'cs.CV',
    'stat.ML', 'q-fin.EC', 'econ.EM', 'cs.CE', 'stat.CO', 'math.PR', 'q-bio.NC', 'math.AP',
    'cs.OS', 'cs.NI', 'cs.IT', 'cs.PL', 'cs.GT', 'cs.DM', 'math.IT', 'cs.SE', 'cs.RO',
    'stat.TH', 'cs.DB', 'math.ST', 'q-bio.GN', 'q-fin.PM', 'q-bio.TO', 'math.GR', 'cs.IR'
]

# Reorder columns based on the desired order
predicted_df = predicted_df[desired_column_order]


# Save the results to a new CSV file
predicted_df.to_csv('lohit.csv', index=False)


In [29]:
len(desired_column_order)

58

In [30]:
pd.read_csv("submissions.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'submissions.csv'

In [ ]:


# Load the CSV files
df1 = pd.read_csv('/content/submission1.csv')
df2 = pd.read_csv('/content/submissions (11).csv')

# Ensure the column names match
if list(df1.columns) == list(df2.columns):
    # Compare the DataFrames
    concatenated = pd.concat([df1, df2]).drop_duplicates(keep=False)
    #differences = df1.compare(df2)

    num_differences = len(concatenated) / 2  # Divide by 2 because each difference appears twice in the concatenation
    print(f"Number of different elements (rows): {int(num_differences)}")
else:
    print("The column names do not match.")

#     # Check if there are any differences
#     if not differences.empty:
#         print("Differences found:")
#         print(differences)
#     else:
#         print("The files are identical in terms of data.")
# else:
#     print("The column names do not match.")